# HSE 2022: Mathematical Methods for Data Analysis

## Homework 2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import datasets
from sklearn.datasets import load_boston
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLSResults
from math import sqrt
import random
import sys

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

sns.set(style="darkgrid")

### Data

For this homework we use Dataset from seaborn on diamonds prices.

In [45]:
data = sns.load_dataset('diamonds')

Y = data.price
X = data.drop(['price'], axis=1)
columns = data.drop(['price'], axis=1).columns
data.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


### Linear regression

#### 0. [0.25 points] Encode categorical variables.

In [47]:
# your code here 
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

def encodeDataSet(data):
  cutValues = list(data['cut'].unique())
  cutValuesToCodes = {value: index for index, value in enumerate(cutValues)}
  data.loc[:, 'cut'] = data.cut.replace(cutValuesToCodes)

  colorValues = list(data['color'].unique())
  colorValuesToCodes = {value: index for index, value in enumerate(colorValues)}
  data.loc[:, 'color'] = data.color.replace(colorValuesToCodes)

  clarityValues = list(data['clarity'].unique())
  clarityValuesToCodes = {value: index for index, value in enumerate(clarityValues)}
  data.loc[:, 'clarity'] = data.clarity.replace(clarityValuesToCodes)

  return data

data = encodeDataSet(data)

Y = data.price
X = data.drop(['price'], axis=1)
data.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,0,0,0,61.5,55.0,326,3.95,3.98,2.43
1,0.21,1,0,1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,2,0,2,56.9,65.0,327,4.05,4.07,2.31
3,0.29,1,1,3,62.4,58.0,334,4.20,4.23,2.63
4,0.31,2,2,0,63.3,58.0,335,4.34,4.35,2.75


#### 1. [0.25 points] Split the data into train and test sets with ratio 80:20 with random_state=17.

In [48]:
# your code here 
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
X_data_train, X_data_test, Y_data_train, Y_data_test = train_test_split(
    X, Y,
    test_size = 0.2,
    random_state = 17
)

#### 2. [1 point] Train models on train data using StatsModels library and apply it to the test set; use $RMSE$ and $R^2$ as the quality measure.

* [`LinearRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html);
* [`Ridge`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) with $\alpha = 0.01$;
* [`Lasso`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) with $\alpha = 0.01$
* [`ElasticNet`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) with $\alpha = 0.01$, $l_{1}$_$ratio = 0.6$

Don't forget to scale the data before training the models with StandardScaler!

In [55]:
# your code here 
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
scaler = StandardScaler()
X_data_train_scaled = scaler.fit_transform(X_data_train)
X_data_test_scaled = scaler.transform(X_data_test)

X_data_train_scaled = sm.add_constant(X_data_train_scaled)
model = sm.OLS(Y_data_train, X_data_train_scaled)
result = model.fit()
display(result.summary())

print("\n\n")

X_data_test_scaled = sm.add_constant(X_data_test_scaled)
Y_predict = result.predict(X_data_test_scaled)

print("RMSE:\n")
display(mean_squared_error(Y_data_test, Y_predict))
print("\n\nR^2:\n")
display(r2_score(Y_data_test, Y_predict))

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.874
Model:                            OLS   Adj. R-squared:                  0.874
Method:                 Least Squares   F-statistic:                 3.326e+04
Date:                Sat, 15 Oct 2022   Prob (F-statistic):               0.00
Time:                        18:31:29   Log-Likelihood:            -3.7423e+05
No. Observations:               43152   AIC:                         7.485e+05
Df Residuals:                   43142   BIC:                         7.486e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3928.6813      6.802    577.592      0.000    3915.350    3942.013
x1          4970.8214     32.497    152.965      0.000    4907.128    5034.515
x2          -100.6240      7.787    -12.921      0.000    -115.887     -85.361
x3           181.0372      6.829     26.510      0.000     167.652     194.422
x4           452.8121      7.226     62.661      0.000     438.648     466.976
x5          -227.6577      8.549    -26.629      0.000    -244.414    -210.901
x6          -133.5915      8.165    -16.361      0.000    -149.596    -117.587
x7         -1273.9011     50.821    -25.067      0.000   -1373.510   -1174.292
x8            78.1147     30.943      2.525      0.012      17.467     138.763
x9            50.6019     31.078      1.628      0.103     -10.311     111.515
==============================================================================
Omnibus:                     9391.792   Durbin-Watson:                   2.008
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           434099.928
Skew:                          -0.007   Prob(JB):                         0.00
Kurtosis:                      18.538   Cond. No.                         17.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""




RMSE:



2097470.912983042



R^2:



0.8704754690827957

#### 3. [1 point] Explore the values of the parameters of the resulting models and compare the number of zero weights in them. Comment on the significance of the coefficients, overal model significance and other related factors from the results table

In [12]:
# your code here 
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

#### 4. [1 point] Implement one of the elimination algorithms that were described in the Seminar_4 (Elimination by P-value, Forward elimination, Backward elimination), make conclusions.

In [13]:
# your code here 
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

#### 5. [1 point] Find the best (in terms of RMSE) $\alpha$ for Lasso regression using cross-validation with 4 folds. You must select values from range $[10^{-4}, 10^{3}]$.

In [14]:
# your code here
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

## Gradient descent

#### 6. [3.5 points] Implement a Ridge regression model for the MSE loss function, trained by gradient descent.

All calculations must be vectorized, and python loops can only be used for gradient descent iterations. As a stop criterion, you must use (simultaneously):

* checking for the Absolute-value norm of the weight difference on two adjacent iterations (for example, less than some small number of the order of $10^{-6}$, set by the `tolerance` parameter);
* reaching the maximum number of iterations (for example, 10000, set by the `max_iter` parameter).

You need to implement:

* Full gradient descent:

$$
w_{k + 1} = w_{k} - \eta_{k} \nabla_{w} Q(w_{k}).
$$

* Stochastic Gradient Descent:

$$
w_{k + 1} = w_{k} - \eta_{k} \nabla_{w} q_{i_{k}}(w_{k}).
$$

$\nabla_{w} q_{i_{k}}(w_{k}) \, $ is the estimate of the gradient over the batch of objects selected randomly.

* Momentum method:

$$
h_0 = 0, \\
h_{k + 1} = \alpha h_{k} + \eta_k \nabla_{w} Q(w_{k}), \\
w_{k + 1} = w_{k} - h_{k + 1}.
$$

* Adagrad method:

$$
G_0 = 0, \\
G_{k + 1} = G_{k} + (\nabla_{w} Q(w_{k+1}))^2, \\
w_{k + 1} = w_{k} - \eta * \frac{\nabla_{w} Q(w_{k+1})}{\sqrt{G_{k+1} + \epsilon}}.
$$



To make sure that the optimization process really converges, we will use the `loss_history` class attribute. After calling the `fit` method, it should contain the values of the loss function for all iterations, starting from the first one (before the first step on the anti-gradient).

You need to initialize the weights with a random vector from normal distribution. The following is a template class that needs to contain the code implementing all variations of the models.

In [ ]:
from sklearn.base import BaseEstimator

class LinReg(BaseEstimator):
    def __init__(self, delta=1.0, gd_type='Momentum', 
                 tolerance=1e-4, max_iter=1000, w0=None, eta=1e-2, alpha=1e-3):
        """
        gd_type: str
            'GradientDescent', 'StochasticDescent', 'Momentum', 'Adagrad'
        delta: float
            proportion of object in a batch (for stochastic GD)
        tolerance: float
            for stopping gradient descent
        max_iter: int
            maximum number of steps in gradient descent
        w0: np.array of shape (d)
            init weights
        eta: float
            learning rate
        alpha: float
            momentum coefficient
        reg_cf: float
            regularization coefficient
        epsilon: float
            numerical stability
        """
        
        self.delta = delta
        self.gd_type = gd_type
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.eta = eta
        self.loss_history = None # list of loss function values at each training iteration
    
    def fit(self, X, y):
        """
        X: np.array of shape (l, d)
        y: np.array of shape (l)
        ---
        output: self
        """
        self.loss_history = []

        return self
    
    def predict(self, X):
        if self.w is None:
            raise Exception('Not trained yet')
        
        pass
    
    def calc_gradient(self, X, y):
        """
        X: np.array of shape (l, d) (l can be equal to 1 if stochastic)
        y: np.array of shape (l)
        ---
        output: np.array of shape (d)
        """
        pass

    def calc_loss(self, X, y):
        """
        X: np.array of shape (l, d)
        y: np.array of shape (l)
        ---
        output: float 
        """ 
        pass

#### 7. [1 points] Train and validate "hand-written" models on the same data, and compare the quality with the Sklearn or StatsModels methods. Investigate the effect of the `max_iter` and `alpha` parameters on the optimization process. Is it consistent with your expectations?

In [ ]:
# your code here 
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

#### 8. [1 points] Plot graphs (on the same picture) of the dependence of the loss function value on the iteration number for Full GD, SGD, Momentum and Adagrad. Draw conclusions about the rate of convergence of various modifications of gradient descent.

Don't forget about what *beautiful* graphics should look like!

In [ ]:
# your code here 
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ